<a href="https://colab.research.google.com/github/lakshanravi/Conversational-RAG-app/blob/main/05.rag_application_langchain_openai/05_conversational_rag_langchain_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conversational RAG Application** with LangChain and Google GenAI LLM

In [1]:
# Install the necessary packages
!pip install langchain -qU
!pip install -U langchain langchain-google-genai
!pip install langchain-chroma -qU
!pip install langchain_community -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0

In [2]:
import os
from google.colab import userdata

### Initialize Google GenAI LLM

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Set OpenAI API key
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0
)

### Initialize Embedding Model

In [4]:
#for embedding the inputs
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

### Load PDF Document

In [5]:
!pip install pypdf -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 10.4 MB/s eta 0:00:00


In [6]:
#To upload the pdf file from my device. Through the imported pdf, we can store the farmscape data in vector DB.
from google.colab import files

# Upload file from your computer
uploaded = files.upload()


Saving FARMSCAP.pdf to FARMSCAP.pdf


In [7]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("/content/FARMSCAP.pdf")

docs = loader.load()

In [8]:
len(docs)

7

### Split Documents into Chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

20

### Create Vector Store and Retriever

In [ ]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [ ]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

### Define Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt
system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

### Create Retrieval-Augmented Generation (RAG) Chain

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, qa_chain)

### Invoke RAG Chain with Example Questions

In [ ]:
response = rag_chain.invoke({"input": "who is codeprolk"})
response["answer"]

'CodePRO LK is a dynamic educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It offers a diverse range of technology-related courses in Sinhala, focusing on programming, data science, and machine learning. CodePRO LK aims to empower Sri Lankans with valuable skills in the tech industry through accessible and high-quality education. The platform continues to evolve and expand its offerings to support its mission of preparing learners for success in the global tech industry.'

In [ ]:
response = rag_chain.invoke({"input": "what is rag architecture"})
response["answer"]

'I\'m not familiar with "rag architecture." It seems to be a term or concept that is not related to the context provided about CodePRO LK and its community engagement, partnerships, and future directions in tech education. If you have more context or details about "rag architecture," I may be able to provide a better answer.'

In [ ]:
response = rag_chain.invoke({"input": "what are the courses codeprolk offer"})
response["answer"]

'CodePRO LK offers a variety of free courses presented in Sinhala, catering to various proficiency levels from beginners to intermediates. Some key courses include Python GUI – Tkinter. In the future, CodePRO LK aims to expand its offerings to cover advanced topics like artificial intelligence, cybersecurity, and advanced data analytics.'

In [ ]:
response = rag_chain.invoke({"input": "can you list down"})
response["answer"]

'List down what specifically? Please provide more details so I can assist you accurately.'

## Add Chat History

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Define the contextualize system prompt
contextualize_system_prompt = (
    "using chat history and the latest user question, just reformulate question if needed and otherwise return it as is"
)

# Create the contextualize prompt template
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_prompt
)

#### Create History-Aware RAG Chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [ ]:
# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the history aware RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

#### Manage Chat Session History

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the store for session histories
store = {}

# Function to get the session history for a given session ID
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the conversational RAG chain with session history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

#### Invoke Conversational RAG Chain with Example Questions

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "who is codeprolk"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'CodePRO LK is a dynamic educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It offers a diverse range of technology-related courses in Sinhala, focusing on programming, data science, and machine learning. CodePRO LK aims to empower Sri Lankans with valuable skills in the tech industry through accessible and high-quality education. The platform continues to evolve and expand its offerings to support its mission of preparing learners for success in the global tech industry. Additionally, CodePRO LK engages its community through various events like webinars, live coding sessions, hackathons, and tech talks to provide networking opportunities and practical experience. The platform also collaborates with educational institutions, tech companies, and industry experts to enhance its content and resources, ensuring learners are well-prepared for real-world challenges.'

In [ ]:
store

{'101': InMemoryChatMessageHistory(messages=[HumanMessage(content='who is codeprolk'), AIMessage(content='CodePRO LK is a dynamic educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It offers a diverse range of technology-related courses in Sinhala, focusing on programming, data science, and machine learning. CodePRO LK aims to empower Sri Lankans with valuable skills in the tech industry through accessible and high-quality education. The platform continues to evolve and expand its offerings to support its mission of preparing learners for success in the global tech industry. Additionally, CodePRO LK engages its community through various events like webinars, live coding sessions, hackathons, and tech talks to provide networking opportunities and practical experience. The platform also collaborates with educational institutions, tech companies, and industry experts to enhance its content and resources, ensuring learners are well-prepared for real-world chall

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "what is rag architecture"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'I\'m sorry, but I don\'t have information on "rag architecture." It seems to be a specific term or concept that is not related to the context provided about CodePRO LK. If you have any other questions or need clarification on a different topic, feel free to ask!'

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "what are the courses codeprolk offer"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

"CodePRO LK offers a variety of technology-related courses in Sinhala to empower learners with valuable skills in programming, data science, and machine learning. Some of the key courses offered by CodePRO LK include:\n\n1. Python GUI – Tkinter: This course covers the essentials of creating graphical user interfaces using Python's Tkinter library.\n\nCodePRO LK aims to cater to learners of all proficiency levels, from beginners to intermediates, ensuring that individuals at different stages can benefit from the courses. Additionally, the platform plans to expand its course offerings in the future to cover more advanced topics and emerging technologies such as artificial intelligence, cybersecurity, and advanced data analytics."

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "can you list down"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

"I apologize for the confusion earlier. Here is a list of some of the courses offered by CodePRO LK:\n\n1. Python GUI – Tkinter: This course covers the essentials of creating graphical user interfaces using Python's Tkinter library.\n\nPlease note that CodePRO LK aims to expand its course offerings in the future to cover more advanced topics and emerging technologies such as artificial intelligence, cybersecurity, and advanced data analytics."